In [1]:
# movie review dataset homepage: http://www.cs.cornell.edu/people/pabo/movie-review-data/
# Download the dataset used here: http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz

# In this code
## Step1 - Generate vocabulary from the raw text data
## Step2 - basic data preprocessing + neural network

In [ ]:
import nltk

nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

In [15]:
# STEP 1 - Generate Vocabulary (word, count)

import string
from collections import Counter
from os import listdir

neg_reviews_folder = "review_polarity/txt_sentoken/neg/"
pos_reviews_folder = "review_polarity/txt_sentoken/pos/"

# convert each raw text to cleaned token list
def clean_text(raw_text_path):
    with open(raw_text_path) as text_in:
        raw_text = text_in.read()
        tokens = raw_text.split()
        # remove punctuation
        tokens = [w.strip(string.punctuation) for w in tokens]
        # only keep alphabetic non-stop words
        tokens = [w for w in tokens if w.isalpha()==True and w not in stopwords]
        # remove short words
        tokens = [w for w in tokens if len(w) >= 3]
    return tokens

vocabCounter = Counter()

for f_name in listdir(neg_reviews_folder):
    if f_name.startswith('cv7'):
        neg_tokens = clean_text(neg_reviews_folder + f_name)
        vocabCounter.update(neg_tokens)
           
for f_name in listdir(pos_reviews_folder):
    if f_name.startswith('cv9'):
        pos_tokens = clean_text(pos_reviews_folder + f_name)
        vocabCounter.update(pos_tokens)

In [17]:
print len(vocabCounter)
print vocabCounter.most_common(79)

13153
[('film', 944), ('movie', 646), ('one', 541), ('like', 337), ('even', 269), ('good', 242), ('time', 235), ('story', 215), ('also', 214), ('would', 209), ('much', 204), ('get', 202), ('characters', 195), ('character', 191), ('see', 188), ('two', 188), ('well', 187), ('films', 184), ('way', 173), ('could', 169), ('really', 166), ('make', 166), ('first', 165), ('people', 162), ('plot', 160), ('little', 154), ('movies', 149), ('bad', 148), ('life', 141), ('many', 137), ('scene', 137), ('scenes', 134), ('another', 133), ('action', 132), ('new', 132), ('best', 128), ('never', 125), ('know', 123), ('director', 119), ('great', 117), ('something', 114), ('original', 111), ('man', 111), ('enough', 110), ('end', 110), ('back', 109), ('makes', 109), ('better', 109), ('however', 108), ('made', 108), ('world', 104), ('john', 104), ('going', 103), ('star', 101), ('love', 98), ('still', 98), ('things', 96), ('audience', 95), ('nothing', 94), ('say', 93), ('actually', 93), ('seems', 93), ('work',